<a href="https://colab.research.google.com/github/mtrefilek/cs762/blob/main/resnet50_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import torch
import os
cwd = os.getcwd().replace('\\','/')

import numpy as np
import pickle
from copy import deepcopy
from torchvision import datasets
from glob import glob, iglob
from PIL import Image
import torchvision.transforms as transforms
import torchvision.models as models
from torch.autograd import Variable

In [49]:
DSET_NAME = 'FMNIST' #('MNIST', 'FMNIST', 'CIFAR10', 'CIFAR100', 'PlantDisease', 'EuroSAT', 'ChestXRay') 'ISIC2018', 'TinyImageNet' are not yet implemented
PRETRAINED_MODEL_NAME = 'RESNET50'

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [50]:
# Load the pretrained model
model = models.resnet50(pretrained=True)
# Use the model object to select the desired layer
layer = model._modules.get('avgpool')
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [51]:
# Set model to evaluation mode
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [52]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

#import torchvision
transform=transforms.Compose([
    # you can add other transformations in this list
    transforms.ToTensor()
])
dset_path = cwd + '/dataset'
feature_path = cwd+'/extracted_features/'

if DSET_NAME=='MNIST': # MNIST
    mnist_train = datasets.MNIST(root=dset_path, train=True, download=True)#, transform=transform)
    mnist_test = datasets.MNIST(root=dset_path, train=False, download=True)#, transform=transform)
    imgs_tr, imgs_tst, labels_tr, labels_tst = [], [], [], []
    for (img, label) in mnist_train:
        img = Variable(transform(img.convert(mode='RGB')))
        imgs_tr.append(img)
        labels_tr.append(label)
    for (img, label) in mnist_test:
        img = Variable(transform(img.convert(mode='RGB')))
        imgs_tst.append(img)
        labels_tst.append(label)
    classnames = [str(a) for a in range(10)]
    
elif DSET_NAME=='FMNIST': # Fashion-MNIST
    trainset = datasets.FashionMNIST(root=dset_path, train=True, download=True)
    testset = datasets.FashionMNIST(root=dset_path, train=False, download=True)
    imgs_tr, imgs_tst, labels_tr, labels_tst = [], [], [], []
    for (img, label) in trainset:
        img = Variable(transform(img.convert(mode='RGB')))
        imgs_tr.append(img)
        labels_tr.append(label)
    for (img, label) in testset:
        img = Variable(transform(img.convert(mode='RGB')))
        imgs_tst.append(img)
        labels_tst.append(label)
    classnames = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

elif DSET_NAME=='CIFAR10': # CIFAR-10
    trainset = datasets.CIFAR10(root=dset_path, train=True, download=True)
    testset = datasets.CIFAR10(root=dset_path, train=False, download=True)
    imgs_tr, imgs_tst, labels_tr, labels_tst = [], [], [], []
    for (img, label) in trainset:
        img = Variable(transform(img.convert(mode='RGB')))
        imgs_tr.append(img)
        labels_tr.append(label)
    for (img, label) in testset:
        img = Variable(transform(img.convert(mode='RGB')))
        imgs_tst.append(img)
        labels_tst.append(label)
    meta = pickle.load( open(dset_path+'/cifar-10-batches-py/batches.meta','rb') )
    classnames = meta['label_names']

elif DSET_NAME=='CIFAR100': # CIFAR-100
    trainset = datasets.CIFAR100(root=dset_path, train=True, download=True)
    testset = datasets.CIFAR100(root=dset_path, train=False, download=True)
    imgs_tr, imgs_tst, labels_tr, labels_tst = [], [], [], []
    for (img, label) in trainset:
        img = Variable(transform(img.convert(mode='RGB')))
        imgs_tr.append(img)
        labels_tr.append(label)
    for (img, label) in testset:
        img = Variable(transform(img.convert(mode='RGB')))
        imgs_tst.append(img)
        labels_tst.append(label)
    meta = pickle.load( open(dset_path+'/cifar-100-python/meta','rb') )
    classnames = meta['fine_label_names']

In [53]:
### Extract Features
if DSET_NAME in ('MNIST', 'FMNIST', 'CIFAR10', 'CIFAR100'):
    n_tr = len(imgs_tr)
    feature_matrix_tr = np.zeros((1, 2048))
    for img_batch in batch(imgs_tr, n=256):
        inputs = torch.stack(img_batch).to(device)
        img_features_tr = feature_extractor(inputs).squeeze().cpu().detach().numpy()
        feature_matrix_tr = np.concatenate((feature_matrix_tr, img_features_tr), axis=0)
        print('Extracting Training Features: {0:.2f}% done'.format(100*len(feature_matrix_tr[1:])/n_tr) )
    feature_matrix_tr = feature_matrix_tr[1:]

    n_cls = np.max(labels_tr)+1
    labels_tr = np.array(labels_tr)
    feature_matrices_tr = []
    for i in range(n_cls):
        feature_matrices_tr.append(feature_matrix_tr[labels_tr==i])

    n_tst = len(imgs_tst)
    feature_matrix_tst = np.zeros((1, 2048))
    for img_batch in batch(imgs_tst, n=256):
      inputs = torch.stack(img_batch).to(device)
      img_features_tst = feature_extractor(inputs).squeeze().cpu().detach().numpy()
      feature_matrix_tst = np.concatenate((feature_matrix_tst, img_features_tst), axis=0)
      print('Extracting Test Features: {0:.2f}% done'.format(100*len(feature_matrix_tst[1:])/n_tst) )
    feature_matrix_tst = feature_matrix_tst[1:]

    n_cls = np.max(labels_tst)+1
    labels_tst = np.array(labels_tst)
    feature_matrices_tst = []
    for i in range(n_cls):
        feature_matrices_tst.append(feature_matrix_tst[labels_tst==i])
    


Extracting Training Features: 0.43% done
Extracting Training Features: 0.85% done
Extracting Training Features: 1.28% done
Extracting Training Features: 1.71% done
Extracting Training Features: 2.13% done
Extracting Training Features: 2.56% done
Extracting Training Features: 2.99% done
Extracting Training Features: 3.41% done
Extracting Training Features: 3.84% done
Extracting Training Features: 4.27% done
Extracting Training Features: 4.69% done
Extracting Training Features: 5.12% done
Extracting Training Features: 5.55% done
Extracting Training Features: 5.97% done
Extracting Training Features: 6.40% done
Extracting Training Features: 6.83% done
Extracting Training Features: 7.25% done
Extracting Training Features: 7.68% done
Extracting Training Features: 8.11% done
Extracting Training Features: 8.53% done
Extracting Training Features: 8.96% done
Extracting Training Features: 9.39% done
Extracting Training Features: 9.81% done
Extracting Training Features: 10.24% done
Extracting Trai

In [54]:
### Save Features
if DSET_NAME in ('MNIST', 'FMNIST', 'CIFAR10', 'CIFAR100'):
    fname_tr = DSET_NAME+'_'+PRETRAINED_MODEL_NAME+'_train.npz'
    fname_tst = DSET_NAME+'_'+PRETRAINED_MODEL_NAME+'_test.npz'

    np.savez(feature_path + fname_tr, feature_matrices = feature_matrices_tr, classnames = classnames)
    np.savez(feature_path + fname_tst, feature_matrices = feature_matrices_tst, classnames = classnames)